In [4]:
import torch 
import torch.nn as nn
import torch.optim as optim 
import torchvision.transforms as transforms
import torchvision.models as models
from torch.utils.data import Dataset, DataLoader
from torch.cuda.amp import autocast, GradScaler
from PIL import Image
import os
import time
from tqdm import tqdm

In [5]:
def get_device_info():
    """Set up and return device information"""
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    print(f"Using device: {device}")
    if torch.cuda.is_available():
        print(f"GPU: {torch.cuda.get_device_name(0)}")
        print(f"Total GPU Memory: {torch.cuda.get_device_properties(0).total_memory/1e9:.2f} GB")
    return device   

In [6]:
def get_config():
    """Return configuration parameters"""
    return {
        "IMG_SIZE": 224,  # ResNet requires 224x224 images
        "BATCH_SIZE": 32,  # Smaller batch size for HDD
        "EPOCHS": 10,
        "DATA_DIR": "B:\\Projects\\lung_cancer_detection\\images",
        "NUM_WORKERS": 0,  # No parallel workers for HDD
        "LEARNING_RATE": 0.0001,
        "WEIGHT_DECAY": 1e-4,
    }

In [7]:
class LungCancerDataset(Dataset):
    def __init__(self, root_dir, transform=None, max_samples_per_class=None):
        self.root_dir = root_dir
        self.transform = transform
        self.samples = []  # (path, label) pairs
        
        # Get class names
        self.classes = sorted(os.listdir(root_dir))
        print(f"Found {len(self.classes)} classes: {self.classes}")
        
        # Load paths only (no images yet)
        for class_idx, class_name in enumerate(self.classes):
            class_path = os.path.join(root_dir, class_name)
            files = os.listdir(class_path)
            if max_samples_per_class:
                files = files[:max_samples_per_class]
            for img_name in files:
                img_path = os.path.join(class_path, img_name)
                self.samples.append((img_path, class_idx))
        
        print(f"Dataset initialized with {len(self.samples)} samples")
    
    def __len__(self):
        return len(self.samples)
    
    def __getitem__(self, idx):
        img_path, label = self.samples[idx]
        config = get_config()
        
        try:
            # Simple image loading
            image = Image.open(img_path).convert('RGB')
            
            if self.transform:
                image = self.transform(image)
            
            return image, label
        except Exception as e:
            print(f"Error loading image {img_path}: {e}")
            # Return a blank tensor as fallback
            return torch.zeros(3, config["IMG_SIZE"], config["IMG_SIZE"]), label

In [8]:
def get_data_loaders():
    """Prepare datasets and dataloaders with augmentation"""
    config = get_config()
    
    # Enhanced augmentation for training
    train_transform = transforms.Compose([
        transforms.RandomResizedCrop(config["IMG_SIZE"], scale=(0.8, 1.0)),
        transforms.RandomHorizontalFlip(),
        transforms.RandomVerticalFlip(),
        transforms.RandomRotation(15),
        transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.1, hue=0.1),
        transforms.RandomAffine(degrees=0, translate=(0.1, 0.1), scale=(0.9, 1.1)),
        transforms.GaussianBlur(kernel_size=3, sigma=(0.1, 2.0)),
        transforms.RandomPerspective(distortion_scale=0.2, p=0.5),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
        transforms.RandomErasing(p=0.2, scale=(0.02, 0.2)),
    ])
    
    test_transform = transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(config["IMG_SIZE"]),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
    ])
    
    # Get all files from the data directory
    all_classes = sorted(os.listdir(config["DATA_DIR"]))
    all_files = []
    for class_idx, class_name in enumerate(all_classes):
        class_path = os.path.join(config["DATA_DIR"], class_name)
        class_files = [(os.path.join(class_path, f), class_idx) for f in os.listdir(class_path)]
        all_files.extend(class_files)
    
    # Split into train and test (80/20 split)
    import random
    random.seed(42)  # For reproducibility
    random.shuffle(all_files)
    
    split_idx = int(len(all_files) * 0.8)
    train_files = all_files[:split_idx]
    test_files = all_files[split_idx:]
    
    print(f"Total files: {len(all_files)}")
    print(f"Training files: {len(train_files)}")
    print(f"Test files: {len(test_files)}")
    
    # Create custom datasets
    class CustomDataset(Dataset):
        def __init__(self, file_list, transform=None):
            self.file_list = file_list
            self.transform = transform
            self.classes = all_classes
            
        def __len__(self):
            return len(self.file_list)
        
        def __getitem__(self, idx):
            img_path, label = self.file_list[idx]
            
            try:
                image = Image.open(img_path).convert('RGB')
                if self.transform:
                    image = self.transform(image)
                return image, label
            except Exception as e:
                print(f"Error loading image {img_path}: {e}")
                return torch.zeros(3, config["IMG_SIZE"], config["IMG_SIZE"]), label
    
    # Create datasets
    train_dataset = CustomDataset(train_files, transform=train_transform)
    test_dataset = CustomDataset(test_files, transform=test_transform)
    
    # Create dataloaders
    train_loader = DataLoader(
        train_dataset, 
        batch_size=config["BATCH_SIZE"], 
        shuffle=True,
        num_workers=config["NUM_WORKERS"],
        pin_memory=True
    )
    
    test_loader = DataLoader(
        test_dataset, 
        batch_size=config["BATCH_SIZE"], 
        shuffle=False,
        num_workers=config["NUM_WORKERS"],
        pin_memory=True
    )
    
    return train_dataset, test_dataset, train_loader, test_loader



In [9]:
def create_resnet18_model(num_classes):
    """Create and return a pre-trained ResNet-18 model with customized final layer"""
    # Load pre-trained ResNet-18
    model = models.resnet18(weights='IMAGENET1K_V1')
    
    # Freeze early layers to speed up training
    for param in list(model.parameters())[:-2*4]:  # Freeze all but final block + FC
        param.requires_grad = False
    
    # Replace the final fully connected layer
    in_features = model.fc.in_features
    model.fc = nn.Linear(in_features, num_classes)
    
    return model

In [10]:
def setup_model(dataset):
    """Set up and return model, criterion, optimizer, and scheduler"""
    config = get_config()
    num_classes = len(dataset.classes)
    device = get_device_info()
    
    model = create_resnet18_model(num_classes)
    model = model.to(device)
    
    # Count trainable parameters
    trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
    total_params = sum(p.numel() for p in model.parameters())
    print(f"Model loaded with {trainable_params:,}/{total_params:,} trainable parameters")
    
    # Optimization setup
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(filter(lambda p: p.requires_grad, model.parameters()),lr=config["LEARNING_RATE"],weight_decay = config["WEIGHT_DECAY"])
    scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.5, patience=3, verbose=True)
    
    return model, criterion, optimizer, scheduler


In [11]:
def train_one_epoch(model, train_loader, criterion, optimizer, device, scaler):
    """Train for one epoch and return statistics"""
    model.train()
    start_time = time.time()
    running_loss = 0.0
    running_corrects = 0
    samples_count = 0
    
    # Use tqdm for training progress
    train_pbar = tqdm(train_loader, desc=f"Training", unit="batch")
    
    for inputs, labels in train_pbar:
        batch_start = time.time()
        
        # Move to device
        inputs = inputs.to(device, non_blocking=True)
        labels = labels.to(device, non_blocking=True)
        
        # Zero gradients
        optimizer.zero_grad(set_to_none=True)
        
        # Forward pass with mixed precision
        with autocast():
            outputs = model(inputs)
            loss = criterion(outputs, labels)
        
        # Backward pass with scaling
        scaler.scale(loss).backward()
        scaler.step(optimizer)
        scaler.update()
        
        # Statistics
        running_loss += loss.item() * inputs.size(0)
        _, preds = torch.max(outputs, 1)
        running_corrects += torch.sum(preds == labels).item()
        samples_count += inputs.size(0)
        
        # Update progress bar with current loss
        batch_time = time.time() - batch_start
        train_pbar.set_postfix({
            'loss': f"{loss.item():.4f}",
            'speed': f"{inputs.size(0)/batch_time:.1f} img/s"
        })
    
    # Epoch statistics
    epoch_loss = running_loss / samples_count
    epoch_acc = running_corrects / samples_count * 100
    epoch_time = time.time() - start_time
    
    return epoch_loss, epoch_acc, epoch_time

def validate(model, test_loader, criterion, device):
    """Validate model and return statistics"""
    model.eval()
    val_running_loss = 0.0
    val_running_corrects = 0
    val_samples_count = 0
    
    # Use tqdm for validation progress
    val_pbar = tqdm(test_loader, desc=f"Validation", unit="batch")
    
    # No gradients needed for validation
    with torch.no_grad():
        for inputs, labels in val_pbar:
            inputs = inputs.to(device, non_blocking=True)
            labels = labels.to(device, non_blocking=True)
            
            # Forward pass
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            
            # Statistics
            val_running_loss += loss.item() * inputs.size(0)
            _, preds = torch.max(outputs, 1)
            val_running_corrects += torch.sum(preds == labels).item()
            val_samples_count += inputs.size(0)
            
            # Update validation progress bar
            val_pbar.set_postfix({'loss': f"{loss.item():.4f}"})
    
    # Validation statistics
    val_epoch_loss = val_running_loss / val_samples_count
    val_epoch_acc = val_running_corrects / val_samples_count * 100
    
    return val_epoch_loss, val_epoch_acc


In [12]:
def print_gpu_stats():
    if torch.cuda.is_available():
        print(f"GPU utilization: {torch.cuda.utilization(0)}%")
        print(f"GPU memory: {torch.cuda.memory_allocated(0)/1e9:.2f}GB / {torch.cuda.get_device_properties(0).total_memory/1e9:.2f}GB")

In [13]:
def plot_training_history(train_losses, train_accs, val_losses, val_accs):
    """Plot training and validation metrics"""
    try:
        import matplotlib.pyplot as plt
        
        epochs = range(1, len(train_losses) + 1)
        
        plt.figure(figsize=(12, 5))
        
        # Plot loss
        plt.subplot(1, 2, 1)
        plt.plot(epochs, train_losses, 'b-', label='Training Loss')
        plt.plot(epochs, val_losses, 'r-', label='Validation Loss')
        plt.title('Training and Validation Loss')
        plt.xlabel('Epochs')
        plt.ylabel('Loss')
        plt.legend()
        
        # Plot accuracy
        plt.subplot(1, 2, 2)
        plt.plot(epochs, train_accs, 'b-', label='Training Accuracy')
        plt.plot(epochs, val_accs, 'r-', label='Validation Accuracy')
        plt.title('Training and Validation Accuracy')
        plt.xlabel('Epochs')
        plt.ylabel('Accuracy')
        plt.legend()
        
        plt.tight_layout()
        plt.savefig('training_history.png')
        plt.close()
        print("Training history plot saved as 'training_history.png'")
    except ImportError:
        print("Matplotlib not available. Skipping plot generation.")
    except Exception as e:
        print(f"Error generating plots: {e}")


In [14]:
def train_model(model, train_loader, test_loader, criterion, optimizer, scheduler, device, epochs=5):
    """Main training loop"""
    model.to(device)
    scaler = GradScaler()
    
    print("\n===== Training Started =====")
    
    best_val_acc = 0.0
    patience = 5
    patience_counter = 0

    train_losses, train_accs = [], []
    val_losses, val_accs = [], []
    
    for epoch in range(epochs):
        print(f"\nEpoch {epoch+1}/{epochs}")
        
        # Training phase
        epoch_loss, epoch_acc, epoch_time = train_one_epoch(
            model, train_loader, criterion, optimizer, device, scaler
        )
        print(f"Train - Loss: {epoch_loss:.4f}, Acc: {epoch_acc:.2f}%, Time: {epoch_time:.2f}s")
        
        # Validation phase
        val_epoch_loss, val_epoch_acc = validate(model, test_loader, criterion, device)
        print(f"Validation - Loss: {val_epoch_loss:.4f}, Acc: {val_epoch_acc:.2f}%")

        train_losses.append(epoch_loss)
        train_accs.append(epoch_acc)
        val_losses.append(val_epoch_loss)
        val_accs.append(val_epoch_acc)
        
        # Update learning rate based on validation loss
        scheduler.step(val_epoch_loss)
        
        # Save best model
        if val_epoch_acc > best_val_acc:
            best_val_acc = val_epoch_acc
            patience_counter = 0
            print(f"Saving best model with accuracy: {val_epoch_acc:.2f}%")
            torch.save(model.state_dict(), 'best_resnet18_model.pth')
        else:
            patience_counter += 1
            if patience_counter >= patience:
                print(f"Early stopping triggered after {epoch+1} epochs")
                break
        
        # Memory cleanup
        torch.cuda.empty_cache()
    
    print(f"\n===== Training Complete =====")
    print(f"Best validation accuracy: {best_val_acc:.2f}%")
    return model

In [15]:
# ===== MAIN FUNCTION =====
def main():
    """Main function to execute the training pipeline"""
    # Get device info
    device = get_device_info()
    
    # Load data
    print("\n===== Loading Datasets =====")
    train_dataset, test_dataset, train_loader, test_loader = get_data_loaders()
    
    # Setup model and training components
    print("\n===== Setting Up Model =====")
    model, criterion, optimizer, scheduler = setup_model(train_dataset)
    
    # Get configuration
    config = get_config()
    
    # Train the model
    print("\n===== Starting Training =====")
    train_model(
        model=model, 
        train_loader=train_loader, 
        test_loader=test_loader, 
        criterion=criterion, 
        optimizer=optimizer, 
        scheduler=scheduler, 
        device=device, 
        epochs=config["EPOCHS"]
    )
    
    # Load best model for final evaluation
    print("\n===== Final Evaluation =====")
    try:
        model.load_state_dict(torch.load('best_resnet18_model.pth'))
        final_val_loss, final_val_acc = validate(model, test_loader, criterion, device)
        print(f"Final model performance - Loss: {final_val_loss:.4f}, Accuracy: {final_val_acc:.2f}%")
    except Exception as e:
        print(f"Error loading best model: {e}")
    
    print("\n===== Training Complete =====")


if __name__ == "__main__":
    try:
        main()
    except Exception as e:
        print(f"ERROR: {e}")
        import traceback
        traceback.print_exc()



Using device: cpu

===== Loading Datasets =====
Total files: 15000
Training files: 12000
Test files: 3000

===== Setting Up Model =====
Using device: cpu


c:\Users\Aadvik Mazumdar\AppData\Local\Programs\Python\Python311\Lib\site-packages\torch\optim\lr_scheduler.py:62: UserWarning: The verbose parameter is deprecated. Please use get_last_lr() to access the learning rate.
  warnings.warn(
C:\Users\Aadvik Mazumdar\AppData\Local\Temp\ipykernel_19224\91818013.py:4: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  scaler = GradScaler()
c:\Users\Aadvik Mazumdar\AppData\Local\Programs\Python\Python311\Lib\site-packages\torch\amp\grad_scaler.py:132: UserWarning: torch.cuda.amp.GradScaler is enabled, but CUDA is not available.  Disabling.
  warnings.warn(


Model loaded with 4,722,179/11,178,051 trainable parameters

===== Starting Training =====

===== Training Started =====

Epoch 1/10


Training:   0%|          | 0/375 [00:00<?, ?batch/s]C:\Users\Aadvik Mazumdar\AppData\Local\Temp\ipykernel_19224\2616129539.py:23: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with autocast():
c:\Users\Aadvik Mazumdar\AppData\Local\Programs\Python\Python311\Lib\site-packages\torch\amp\autocast_mode.py:266: UserWarning: User provided device_type of 'cuda', but CUDA is not available. Disabling
  warnings.warn(
Training: 100%|██████████| 375/375 [04:45<00:00,  1.31batch/s, loss=0.2914, speed=89.6 img/s]


Train - Loss: 0.2187, Acc: 90.92%, Time: 285.73s


...

Train - Loss: 0.0775, Acc: 97.02%, Time: 322.68s


Validation: 100%|██████████| 94/94 [00:51<00:00,  1.84batch/s, loss=0.1040]
C:\Users\Aadvik Mazumdar\AppData\Local\Temp\ipykernel_19224\518312727.py:34: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to thi

Validation - Loss: 0.0450, Acc: 98.40%

===== Training Complete =====
Best validation accuracy: 98.50%

===== Final Evaluation =====


Validation: 100%|██████████| 94/94 [00:51<00:00,  1.83batch/s, loss=0.0232]

Final model performance - Loss: 0.0437, Accuracy: 98.50%

===== Training Complete =====


In [22]:
import torch
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import f1_score, classification_report, confusion_matrix
import os

def compute_f1_metrics(model, test_loader, criterion, device):
    """
    Compute F1 metrics for model evaluation.
    
    Args:
        model (torch.nn.Module): Trained model
        test_loader (torch.utils.data.DataLoader): Test data loader
        criterion (torch.nn.Module): Loss function
        device (torch.device): Computing device
    
    Returns:
        dict: Evaluation metrics including F1 scores, classification report, and confusion matrix.
    """
    model.eval()
    all_preds, all_labels = [], []
    total_loss = 0.0

    with torch.no_grad():
        for inputs, labels in test_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            total_loss += loss.item()

            _, preds = torch.max(outputs, 1)
            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())

    f1_micro = f1_score(all_labels, all_preds, average='micro')
    f1_macro = f1_score(all_labels, all_preds, average='macro')
    f1_weighted = f1_score(all_labels, all_preds, average='weighted')

    class_names = test_loader.dataset.classes
    report = classification_report(all_labels, all_preds, target_names=class_names)
    conf_matrix = confusion_matrix(all_labels, all_preds)

    return {
        'f1_micro': f1_micro,
        'f1_macro': f1_macro,
        'f1_weighted': f1_weighted,
        'classification_report': report,
        'confusion_matrix': conf_matrix,
        'avg_loss': total_loss / len(test_loader),
        'class_names': class_names
    }

def plot_confusion_matrix(conf_matrix, class_names):
    """
    Plot and save the confusion matrix.

    Args:
        conf_matrix (np.ndarray): Confusion matrix
        class_names (list): Class labels
    """
    plt.figure(figsize=(10, 8))
    sns.heatmap(conf_matrix, annot=True, fmt='d', cmap='Blues',
                xticklabels=class_names, yticklabels=class_names)
    plt.title('Confusion Matrix')
    plt.xlabel('Predicted')
    plt.ylabel('Actual')
    plt.tight_layout()
    plt.savefig('confusion_matrix.png')
    plt.close()

def perform_f1_evaluation(model, test_loader, criterion, device):
    """
    Perform model evaluation using F1 metrics.

    Args:
        model (torch.nn.Module): Trained model
        test_loader (torch.utils.data.DataLoader): Test data loader
        criterion (torch.nn.Module): Loss function
        device (torch.device): Computing device
    """
    print("\n===== Performing Model Evaluation =====")
    
    eval_results = compute_f1_metrics(model, test_loader, criterion, device)

    print("\n--- F1 Scores ---")
    print(f"Micro F1 Score:     {eval_results['f1_micro']:.4f}")
    print(f"Macro F1 Score:     {eval_results['f1_macro']:.4f}")
    print(f"Weighted F1 Score:  {eval_results['f1_weighted']:.4f}")
    print(f"Average Test Loss:  {eval_results['avg_loss']:.4f}")

    print("\n--- Classification Report ---")
    print(eval_results['classification_report'])

    plot_confusion_matrix(eval_results['confusion_matrix'], eval_results['class_names'])

    return eval_results

# ===== UPDATED MAIN FUNCTION =====
def main():
    """Main function to execute the evaluation pipeline"""
    device = get_device_info()

    print("\n===== Loading Datasets =====")
    train_dataset, test_dataset, train_loader, test_loader = get_data_loaders()

    print("\n===== Setting Up Model =====")
    model, criterion, optimizer, scheduler = setup_model(train_dataset)

    model_path = "best_resnet18_model.pth"

    # Load best model if available
    if os.path.exists(model_path):
        print("\n===== Loading Pre-trained Model for Evaluation =====")
        try:
            model.load_state_dict(torch.load(model_path, map_location=device))
            model.to(device)
            model.eval()
        except Exception as e:
            print(f"Error loading best model: {e}")
            return
    
        # Perform F1 score evaluation
        eval_results = perform_f1_evaluation(model, test_loader, criterion, device)
        
        # Save evaluation results
        torch.save(eval_results, "evaluation_results.pth")
    
    else:
        print(f"Error: Model file '{model_path}' not found! Please train the model first.")

if __name__ == "__main__":
    try:
        main()
    except Exception as e:
        print(f"ERROR: {e}")
        import traceback
        traceback.print_exc()


Using device: cpu

===== Loading Datasets =====
Total files: 15000
Training files: 12000
Test files: 3000

===== Setting Up Model =====
Using device: cpu
Model loaded with 4,722,179/11,178,051 trainable parameters

===== Loading Pre-trained Model for Evaluation =====

===== Performing Model Evaluation =====


C:\Users\Aadvik Mazumdar\AppData\Local\Temp\ipykernel_19224\148678676.py:117: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load(model_path, map_


--- F1 Scores ---
Micro F1 Score:     0.9850
Macro F1 Score:     0.9848
Weighted F1 Score:  0.9850
Average Test Loss:  0.0436

--- Classification Report ---
              precision    recall  f1-score   support

    lung_aca       0.99      0.96      0.98       971
      lung_n       1.00      1.00      1.00      1023
    lung_scc       0.97      0.99      0.98      1006

    accuracy                           0.98      3000
   macro avg       0.99      0.98      0.98      3000
weighted avg       0.99      0.98      0.98      3000

